**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [2]:
!pip install keras-flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
  !pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops
from keras.datasets import cifar100

In [5]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [6]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = cifar100.load_data(label_mode='fine')

169001437/169001437 [==============================] - 2s 0us/step


In [7]:
# Преобразуйте метки классов в one_hot формат
y_train = (y_train == np.arange(CLASSES)).astype(np.float32) 
y_val = (y_val == np.arange(CLASSES)).astype(np.float32) 

In [8]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

Зададим архитектуру модели

In [138]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),

    tf.keras.layers.Conv2D(12, kernel_size=(3,3),strides=(2,2),padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

    tf.keras.layers.Conv2D(16, kernel_size=(3,3),strides=(2,2),padding='same'), #16
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512), #521
    #tf.keras.layers.Dense(128), #521
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [126]:
# Задайте архитектуру модели
# model = tf.keras.models.Sequential([
#     tf.keras.Input(shape=[32,32,3]),
#     tf.keras.layers.Conv2D(16, 9),
#     #tf.keras.layers.BatchNormalization(),
#     #tf.keras.layers.Activation('relu'),
#     tf.keras.layers.Activation('softmax')
# ])

In [1]:
# при такой моедли получаем точность 0.3413

# model = tf.keras.models.Sequential([
#     tf.keras.Input(shape=[32,32,3]),

#     tf.keras.layers.Conv2D(16, kernel_size=(4,4),strides=(2,2),padding='same'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Activation('relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

#     tf.keras.layers.Conv2D(16, kernel_size=(3,3),strides=(2,2),padding='same'), #16
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Activation('relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512), #521
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Activation('relu'),
#     tf.keras.layers.Dropout(0.5),

#     tf.keras.layers.Dense(CLASSES),
#     tf.keras.layers.Activation('softmax')
# ])

In [139]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6042e6


In [140]:
flops <= 0.707e6

True

In [141]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 16, 16, 12)        336       
                                                                 
 batch_normalization_82 (Bat  (None, 16, 16, 12)       48        
 chNormalization)                                                
                                                                 
 activation_112 (Activation)  (None, 16, 16, 12)       0         
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 8, 8, 12)         0         
 g2D)                                                            
                                                                 
 conv2d_56 (Conv2D)          (None, 4, 4, 16)          1744      
                                                                 
 batch_normalization_83 (Bat  (None, 4, 4, 16)       

In [142]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [143]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=40
    #epochs=256
)

Epoch 1/40
391/391 [==============================] - 15s 37ms/step - loss: 3.6507 - accuracy: 0.1584 - val_loss: 3.3979 - val_accuracy: 0.1953
Epoch 2/40
391/391 [==============================] - 17s 44ms/step - loss: 3.0696 - accuracy: 0.2465 - val_loss: 3.1251 - val_accuracy: 0.2469
Epoch 3/40
391/391 [==============================] - 14s 36ms/step - loss: 2.8432 - accuracy: 0.2897 - val_loss: 2.9907 - val_accuracy: 0.2731
Epoch 4/40
391/391 [==============================] - 14s 36ms/step - loss: 2.6833 - accuracy: 0.3191 - val_loss: 2.8128 - val_accuracy: 0.3001
Epoch 5/40
391/391 [==============================] - 14s 37ms/step - loss: 2.5611 - accuracy: 0.3441 - val_loss: 2.6238 - val_accuracy: 0.3437
Epoch 6/40
391/391 [==============================] - 14s 37ms/step - loss: 2.4518 - accuracy: 0.3683 - val_loss: 2.5280 - val_accuracy: 0.3629
Epoch 7/40
391/391 [==============================] - 14s 36ms/step - loss: 2.3712 - accuracy: 0.3811 - val_loss: 2.5219 - val_accuracy: